<a href="https://colab.research.google.com/github/aknip/llm-benchmark/blob/main/llm_simple_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai tiktoken litellm pandas openpyxl langchain icecream --quiet

In [ ]:
import json
import os
from getpass import getpass
import psutil
import requests
import textwrap
from icecream import ic
import time
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [ ]:
import openai
from litellm import completion
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v1']['credential'] # my key
os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']
os.environ['ANTHROPIC_API_KEY'] = CREDS['anthropic']['key']['credential']

# Run benchmark

Excel file "llm_benchmark.xlsx" must be in the same folder!

In [ ]:
import tiktoken
import textwrap
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd
import json
from io import StringIO

# load input file
workbook = load_workbook(filename = 'llm_benchmark.xlsx', data_only=True) # data_only=True
worksheet_emails = workbook['Emails'] # workbook.active
worksheet_llm_config = workbook['LLM-Configs']

# create log file (based on input file, sheet 'Results', all other sheets are deleted)
workbook_log = load_workbook(filename = 'llm_benchmark.xlsx')
workbook_log.remove(workbook_log['README'])
workbook_log.remove(workbook_log['Emails'])
workbook_log.remove(workbook_log['LLM-Configs'])
worksheet_log = workbook_log['Results']
while(worksheet_log.max_row > 1): # delete log rows: continuously delete row 2 until there is only first row (header row) left over
  worksheet_log.delete_rows(2)

# create headers dictionaries to access columns
column_email = {}
for column_index, cell in enumerate(worksheet_emails["1"]):
  column_email[cell.value] = column_index + 1
column_llm_config = {}
for column_index, cell in enumerate(worksheet_llm_config["1"]):
  column_llm_config[cell.value] = column_index + 1

# iterate through all lines
for row_index, row1 in enumerate(worksheet_emails):
  if row_index == 12: # > 0  //  == 12
    dateiname = worksheet_emails.cell(row=row_index+1, column=column_email["Dateiname"]).value
    if dateiname != None:
      ic(dateiname)
      prompt_context = worksheet_emails.cell(row=row_index+1, column=column_email["Prompt-Context"]).value
      target_JSON = worksheet_emails.cell(row=row_index+1, column=column_email["Ziel-JSON"]).value
      llm_configs = worksheet_emails.cell(row=row_index+1, column=column_email["LLM-Configs"]).value.split(',')

      for config_index, config in enumerate(llm_configs):
        # search config
        found_flag = False
        for llm_config_index, row2 in enumerate(worksheet_llm_config):
          llm_config_id = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["LLM-Config-ID"]).value
          if llm_config_id.strip() == config.strip():
            found_flag = True
            break
        if found_flag == True:
          ic('LLM-Config found:', dateiname, config)
          llm_config_LLM_Model = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["LLM-Model"]).value

          # map LLM model names for calling by lite-llm
          if llm_config_LLM_Model == 'Mixtral-8x7B':
            llm_config_LLM_Model = 'together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1'

          llm_config_Type = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["Type"]).value
          llm_config_Prompt = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["Prompt"]).value
          if llm_config_Prompt == None:
            llm_config_Prompt = ''
          llm_config_Functions_String = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["Functions"]).value
          if llm_config_Functions_String == None:
            llm_config_Functions = []
          else:
            llm_config_Functions = eval(llm_config_Functions_String)
          llm_config_Response_Format = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["Response-Format"]).value
          prompt_full = llm_config_Prompt + '\n\n' + prompt_context
          prompt_full_tokenlength = len(tiktoken.get_encoding('cl100k_base').encode(prompt_full))
          #for x in prompt_full[:500].split('\n'):
          #  print(textwrap.fill(x, 80))

          #
          # do LLM call here
          #
          start_time = time.time()

          if llm_config_Type == 'Prompt':
            litellm.add_function_to_prompt = False
            response = completion(
              model=llm_config_LLM_Model,
              messages=[{ "content": prompt_full,"role": "user"}],
              #temperature=0.0,
              max_tokens=500
            )
            response_obj = json.dumps(response.json())
            response_txt = response.choices[0].message.content
            response_JSON = ''
            if llm_config_Response_Format == "JSON":
              response_JSON = response_txt
            else:
              if llm_config_Response_Format == "CSV":
                print('Debug CSV:')
                print(response_txt)
                response_JSON = "{dummy}"
                #response_IO = StringIO(response_txt) # [4:] strip away first 4 chars "AI: "
                #response_df = pd.read_csv(response_IO, sep="|")
                #response_JSON_Obj = json.loads(response_df.to_json(orient='table',index=False))['data'][0]
                #response_JSON = json.dumps(response_JSON_Obj) # ,indent=4
                #response_JSON = (response_JSON.encode('latin-1').decode('unicode_escape')) # convert UNICODEs (caused by pd?)

          elif llm_config_Type == 'Function-Call':
            if llm_config_LLM_Model == "claude-2": # =====================================================
              litellm.add_function_to_prompt = True
            else:
              litellm.add_function_to_prompt = False

            response = completion(
              model=llm_config_LLM_Model,
              messages = [{"role": "user", "content": prompt_full}],
              functions=llm_config_Functions
            )

            # Extract function response data
            response_obj = json.dumps(response.json())
            response_txt = response.choices[0].message.content
            response_JSON = ''

            function_found = hasattr(response.choices[0]['message'], 'function_call')
            if function_found == True:
              function_call = response.choices[0]['message']['function_call']
              #function_call_name = function_call.name
              response_JSON = function_call.arguments
            else:
              if llm_config_LLM_Model == "claude-2":
                response_JSON = response.choices[0]['message']['content']
              else:
                response_JSON = {'error': 'no response data found'}

          # remove line breaks from JSON
          response_JSON = json.dumps(json.loads(response_JSON))
          # Fix some escaped characters
          response_JSON = response_JSON.replace('\_', '_')
          # convert UNICODEs (caused by pd?)
          response_JSON = (response_JSON.encode('latin-1').decode('unicode_escape'))

          response_model = response.model
          response_responsetime = float(response._response_ms)/1000
          time.sleep(1)
          #response = "response-obj"
          #response_txt = "dummy text"
          stop_time = time.time()
          elapsed_time = stop_time-start_time

          # log result
          current_date_time = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
          # Time	Email-Name
          log_row = [current_date_time, # Time
                     dateiname,         # Email-Name
                     config.strip(),    # LLM-Config-ID
                     llm_config_LLM_Model, # Input-Model
                     prompt_full,       # Input-Full-Prompt
                     prompt_context,    # Input-Context
                     response_obj,      # Response-Full
                     response_model,    # Response-Model
                     response_responsetime, # Response-Duration
                     elapsed_time,      # Duration-end-to-end
                     response_txt,      # Ergebnis
                     response_JSON,     # Ergebnis-JSON
                     target_JSON         # Ziel-JSON

          ]
          worksheet_log.append(log_row)
        else:
          # LLM-Config not found - log result
          ic('LLM-Config not found:', dateiname, config)
          current_date_time = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
          log_row = [current_date_time, dateiname,'NOT FOUND: ' + config.strip()]
          worksheet_log.append(log_row)

# Save the log file with timestamp
current_date_time = '_' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
current_date_time = "" # for debugging
workbook_log.save('llm_benchmark_log' + current_date_time + '.xlsx')

# Calculate results

In [ ]:
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook

# load log file
workbook_log = load_workbook(filename = 'llm_benchmark_log.xlsx') # data_only=True
worksheet_log = workbook_log['Results']

# create headers dictionaries to access columns
column_log = {}
for column_index, cell in enumerate(worksheet_log["1"]):
  column_log[cell.value] = column_index + 1
ic(column_index)

# iterate through all lines
for log_index, row3 in enumerate(worksheet_log):
  if log_index == 1: # > 0  //  == 1
    log_time = worksheet_log.cell(row=log_index+1, column=column_log["Time"]).value
    ic('found', log_time)

# simple LLM calls for testing

In [ ]:
import openai
import os

client = openai.OpenAI(
  api_key=os.environ.get("TOGETHERAI_API_KEY"),
  base_url='https://api.together.xyz',
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": "Hello, how are you?",
    }
  ],
  model="mistralai/Mixtral-8x7B-Instruct-v0.1",
  max_tokens=1024
)

print(chat_completion.choices[0].message.content)

In [ ]:
# Call via lite-llm

response = completion(
  #model="together_ai/togethercomputer/Llama-2-7B-32K-Instruct",
  model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

In [ ]:
response = completion(
  model="gpt-3.5-turbo",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

In [ ]:
# Function Calling

# see https://litellm.vercel.app/docs/completion/function_call

# via Huggingface?
# https://litellm.vercel.app/docs/providers/huggingface
# https://huggingface.co/Trelis/Mixtral-8x7B-Instruct-v0.1-function-calling-v3
# https://huggingface.co/Trelis/Mistral-7B-Instruct-v0.1-function-calling-v2

# via Anyscale?
# https://docs.litellm.ai/docs/providers/anyscale
# https://www.anyscale.com/blog/anyscale-endpoints-json-mode-and-function-calling-features

import os, litellm
from litellm import completion

# IMPORTANT - Set this to TRUE to add the function to the prompt for Non OpenAI LLMs
litellm.add_function_to_prompt = True

# The real function is not needed for the LLM. It may be called after the LLM call (not in this code!)
def get_current_weather(location):
  if location == "Boston, MA":
    return "The weather is 12F"

functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]

messages = [
    {"role": "user", "content": "What is the weather like in Boston?"}
]

llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-3.5-turbo-1106"
#llm_model = "gpt-4"
#llm_model = "claude-2" # ??? claude answers in response.choices[0]['message']['content']
#llm_model = "together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1"


response = completion(model=llm_model, messages=messages, functions=functions)

function_found = hasattr(response.choices[0]['message'], 'function_call')
if function_found == True:
  function_call = response.choices[0]['message']['function_call']
  function_call_name = function_call.name
  function_call_arguments = function_call.arguments
  print(function_call_arguments)
else:
  if llm_model == "claude-2":
    print(response.choices[0]['message']['content'])
  else:
    print('No function found')

print('\n')
print(response.choices[0]['message']['content'])
print('\n')
print(json.dumps(response.json(), indent=2))

In [ ]:
import os, litellm
from litellm import completion

litellm.add_function_to_prompt = False

functions_string = """[
  {
    "name": "get_risk_data",
    "description": "Daten zum Versicherungsnehmer und seiner Risiken",
    "parameters": {
      "type": "object",
      "properties": {
        "Versicherungssumme": {
          "type": "integer",
          "description": "Versicherungssumme oder Deckungssumme der Versicherung, z. B. 100000"
        },
        "Versicherungsnehmer_Name": {
          "type": "string",
          "description": "Name des versicherten Unternehmens, z. B. 'Testfirma GmbH'"
        },
        "Versicherungsnehmer_Anschrift": {
          "type": "string",
          "description": "Anschrift des versicherten Unternehmens: Strasse, PLZ, Ort, z. B. 'Musterstr. 10, 10000 Berlin'"
        },
        "Versicherungsnehmer_Branche": {
          "type": "string",
          "description": "Branche bzw. Tätigkeitsbereich des versicherten Unternehmens, z. B. 'Textilproduktion'"
        },
        "Versicherungsnehmer_Umsatz": {
          "type": "string",
          "description": "Jahresumsatz des versicherten Unternehmens, z. B. 23000000"
        },
        "Vorschäden_benannt": {
          "type": "string",
          "description": "Informationen zu früheren Schadensfällen, z. B. 'Ja, in 2019 gab es einen Schaden mit folgenden Details'"
        },
      },
      "required": ["Versicherungssumme"]
    }
  }
]
"""

functions=eval(functions_string)

email_test_text = """wir hoffen, Sie hatten einen angenehmen Feiertag.

Zu unserer besteMeiermen Kundenverbindung Zorrsen Beteiligungsgesellschaft mbH bitten wir um Ihre Quotierung zur D&O-Versicherung per 01.01.2024.

Die aktuelle Versicherungssumme beträgt 5,5 Mio. € zu einer Jahresprämie in Höhe von 7.920 € netto.
Der Umsatz im letzten Jahr betrug 78,5 Millionen EUR.

Beigefügt erhalten Sie die bisherigen Vertragsunterlagen sowie ein Organigramm. Versichert werden soll die Zorrsen Beteiligungsgesellschaft mbH als VN und die darunterliegenden Tochtergesellschaften (gelb markiert).

Es gibt einen laufenden D&O-Schadensfall aufgrund einer Inanspruchnahme eines ehemaligen Geschäftsführers, welcher im Jahr 2017 entlassen wurde. Ihm wird vorgeworfen, für ein damaliges Bauvorhaben das Budget deutlich überzogen zu haben.

Für Ihre Rückmeldung vielen Dank im Voraus.
"""

messages = [
    {"role": "user", "content": "Extrahiere die Risikodaten: " + email_test_text}
]

llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-3.5-turbo-1106"
#llm_model = "gpt-4"
#llm_model = "claude-2" # ??? claude answers in response.choices[0]['message']['content']
#llm_model = "together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1"


response = completion(model=llm_model, messages=messages, functions=functions)

function_found = hasattr(response.choices[0]['message'], 'function_call')
if function_found == True:
  function_call = response.choices[0]['message']['function_call']
  function_call_name = function_call.name
  function_call_arguments = function_call.arguments
  print(function_call_arguments)
else:
  if llm_model == "claude-2":
    print(response.choices[0]['message']['content'])
  else:
    print('No function found')

print('\n')
print(response.choices[0]['message']['content'])
print('\n')
print(json.dumps(response.json(), indent=2))

In [ ]:
print(response.choices[0]['message']['content'])

# Anonymisierung

- geht mit GPT-4 und Claude 2.1 mit Abstand am besten
- GPT-4 auf Context 8192 beschränkt


In [ ]:
email1_orig="""
# Email Nachricht:

From: Ralf Löffler - Universal Insurance <r.loeffler@universal-insurance.com>
Sent: Wed, 04 Oct 2023 16:43:22 +0200
To: "Kloeppel; Rolf" <rolf.kloeppel@meiermakler.de>
Cc: Rolf van de Sand - Universal Insurance <m.vandesand@universal-insurance.com>
Subject: Quotierungsanfrage zur D&O-Versicherung, VN: Zorrsen Beteiligungsgesellschaft mbH
-----------------

Hallo Herr Kloeppel,

wir hoffen, Sie hatten einen angenehmen Feiertag.

Zu unserer besteMeiermen Kundenverbindung Zorrsen Beteiligungsgesellschaft mbH bitten wir um Ihre Quotierung zur D&O-Versicherung per 01.01.2024.

Die aktuelle Versicherungssumme beträgt 5,5 Mio. € zu einer Jahresprämie in Höhe von 7.920 € netto.

Beigefügt erhalten Sie die bisherigen Vertragsunterlagen sowie ein Organigramm. Versichert werden soll die Zorrsen Beteiligungsgesellschaft mbH als VN und die darunterliegenden Tochtergesellschaften (gelb markiert).

Es gibt einen laufenden D&O-Schadensfall aufgrund einer Inanspruchnahme eines ehemaligen Geschäftsführers, welcher im Jahr 2017 entlassen wurde. Ihm wird vorgeworfen, für ein damaliges Bauvorhaben das Budget deutlich überzogen zu haben.

Für Ihre Rückmeldung vielen Dank im Voraus.

Mit besten Grüßen

Ralf Löffler
Kundenbetreuer Industrie

T +49 99 190189-18
r.loeffler@universal-insurance.com <mailto:r.loeffler@universal-insurance.com>

Sicherer Dokumentenversand
Ab sofort können Sie uns Ihre Dokumente auch direkt verschlüsselt vom PC & Smartphone über https://www.sicherer-dokumentenversand.de <https://www.sicherer-dokumentenversand.de/>  zukommen lassen.

Universal Insurance Versicherungsmakler GmbH
Karola-Trutschenko-Platz 2 · 20700 Potsdam



www.universal-insurance.com <https://www.universal-insurance.com/impressum>

Unternehmenssitz: Karola-Trutschenko-Platz 2 · 20700 Potsdam
Handelsregister: Amtsgericht Potsdam · HRB 32397
Geschäftsführer: Lutz Lorentzen, Klaas Lollmund, Lutz Ummsen, Hans Berentz, Lorentz H. Gronken, Kai Gronken
Beirat: Lutz Transitz, Prof. Ottmar Halonkeli, Lorentz P. Loppelbaum

Als Versicherungsmakler sind wir nach § 34 d Abs. 1 der Gewerbeordnung tätig und im Vermittlungsregister unter der Registernummer D-KJHG-U7HH-77 eingetragen.

Wir sind klimaneutral! Gerne informieren wir Sie über unsere aktuellen Klimaschutzprojekte.

 <https://www.universal-insurance.com/nachhaltigkeit>      <https://www.linkedin.com/company/global-gruppe-linkedin>    <https://www.xing.com/pages/universalinsurance>   ·   Datenschutzhinweise <https://www.universal-insurance.com/datenschutzerklaerung>    ·   Impressum <https://www.universal-insurance.com/impressum>

Diese E-Mail enthält vertrauliche und/oder rechtlich geschützte Informationen. Wenn Sie nicht der richtige Adressat sind oder diese E-Mail irrtümlich erhalten haben, informieren Sie bitte sofort den Absender und vernichten Sie diese Mail. Das unerlaubte Kopieren sowie die unbefugte Weitergabe dieser E-Mail ist nicht gestattet.

This email may contain confidential and/or privileged information. If you are not the intended recipient (or have received this email in error) please notify the sender immediately and destroy this email. Any unauthorized copying, disclosure or distribution of the material in this email is strictly forbidden.

Ralf Löffler
Kundenbetreuer Industrie

T +49 99 190189-18
r.loeffler@universal-insurance.com <mailto:r.loeffler@universal-insurance.com>
Karola-Trutschenko-Platz 2 · 20700 Potsdam



Handelsregister: Amtsgericht Potsdam · HRB 32397
Geschäftsführer: Lutz Lorentzen, Klaas Lollmund, Lutz Ummsen, Hans Berentz, Lorentz H. Gronken, Kai Gronken
Beirat: Lutz Transitz, Prof. Ottmar Halonkeli, Lorentz P. Loppelbaum
Unternehmenssitz: Karola-Trutschenko-Platz 2 · 20700 Potsdam


Als Versicherungsmakler sind wir nach § 34 d Abs. 1 der Gewerbeordnung tätig und im Vermittlungsregister unter der Registernummer D-KJHG-U7HH-77 eingetragen.

Sicherer Dokumentenversand
Ab sofort können Sie uns Ihre Dokumente auch direkt verschlüsselt vom PC & Smartphone über https://www.sicherer-dokumentenversand.de <https://www.sicherer-dokumentenversand.de>  zukommen lassen.


 <https://signatur.universal-insurance.com/public/global-gruppe.png> 	 <https://www.linkedin.com/company/global-gruppe-linkedin>       <https://www.xing.com/pages/universalinsurance>       <https://www.universal-insurance.com/nachhaltigkeit>  	 <https://signatur.universal-insurance.com/public/wiwo_badge.png>

Datenschutz <https://www.universal-insurance.com/datenschutzerklaerung>    •   Impressum <https://www.universal-insurance.com/impressum>
Diese E-Mail enthält vertrauliche und/oder rechtlich geschützte Informationen. Wenn Sie nicht der richtige Adressat sind oder diese E-Mail irrtümlich erhalten haben, informieren Sie bitte sofort den Absender und vernichten Sie diese E-Mail. Das unerlaubte Kopieren sowie die unbefugte Weitergabe dieser E-Mail ist nicht gestattet.

This email may contain confidential and/or privileged information. If you are not the intended recipient (or have received this email in error) please notify the sender immediately and destroy this email. Any unauthorized copying, disclosure or distribution of the material in this email is strictly forbidden.

# Organigramm:

Zorrsen AG
Schweiz

Zorrsen
Investment AG
Schweiz

Zorrsen Beteilig-
ungsges. GmbH
Deutschland

Zorrsen
Holding GmbH
Deutschland

Testcompany
Menneprom GmbH
Deutschland

Hummsen
Lackfabrik GmbH
Deutschland


# Versicherungsschein:
Versicherungsschein

BBB D&O-Versicherung 098765

Versicherungsnehmer/in

Zorrsen Beteiligungsgesellschaft mbH Juliusweg 9 28374 Zutzenhausen

Versicherungsvermittler

Universal Insurance Versicherungsmakler GmbH Karola-Trutschenko-Platz 2 20700 Potsdam

Versicherungsschein-Nr.

748344 (alt: 43789434)

Vertragsdaten

Vertragsbeginn:

Beginn aktuelle Versicherungsperiode: Ablauf aktuelle Versicherungsperiode:

Versicherungssumme

€ 5.500.000,00

Versicherungsprämie

21.33.2012 12:00 Uhr

01.01.2023 12:00 Uhr

01.01.2024 12:00 Uhr

Die Versicherungsprämie beträgt € 9.900,00 pro Versicherungsjahr (zuzüglich Versicherungsteuer).

Den insgesamt für die o.g. Versicherungsperiode zu zahlenden Versicherungsbeitrag entnehmen Sie bitte der zu diesem Versicherungsschein gesondert erstellten Beitragsrechnung.

Gemäß § 7 Abs. 4 VersStG ist der unten genannte führende Versicherer von den übrigen beteiligten Versicherern der BBB Insurance bestimmt, die Versicherungsteuer auch für diese zu entrichten.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 2 von 8 | BBB-Nr.;: 098765

Vertragsbestimmungen

Die gegenseitigen Rechte und Pflichten richten sich nach den gesetzlichen Bestimmungen der Bundesrepublik Deutschland und

a) den Allgemeinen Bedingungen zur BBB D&O-Versicherung (AVB-BBB 5.0 (Stand: 08/2019))

b

Besondere Vereinbarungen zu den durch die Universal Insurance Versicherungsmakler GmbH vermittelten Policen:

Abwehrkosten nach Eintritt des Versicherungsfalls

§ 2 Ziffer 1.3. (Abwehrkosten nach Eintritt des Versicherungsfalls) der AVB-BBB 5.0 wird wie folgt ergänzt.

Die Abwehrkosten umfassen auch interne Auslagen (durch eigene Mitarbeiter der Versicherungsnehmerin), sofern die nachfolgenden Voraussetzungen vorliegen, wobei der Versicherungsschutz für diese Deckung auf ein Sublimit in Höhe von € 25.000,-- begrenzt ist:

› ›

es handelt sich um einen Drittanspruch; diese Kosten wären aufgrund der hierfür notwendigen Fachexpertise zwingend auch durch externe Berater entstanden, um den Anspruch sachgerecht abzuwehren; › die Versicherungsnehmerin führt den Nachweis über die konkret entstandenen Kosten und › die Versicherungsnehmerin hat vor Entstehung der Kosten die Zustimmung der VOV eingeholt, um den Abrechnungsmodus zu vereinbaren.

Schiedsgerichtsverfahren

§ 2 (Versicherungsleistungen) der AVB-BBB 5.0 wird wie folgt ergänzt:

1.12. Schiedsgerichtsverfahren

Liegt dem Versicherungsfall ein von der Versicherungsnehmerin erhobener Anspruch auf Ersatz eines Vermögensschadens mit einem Streitwert von maximal € 500.000,-- zugrunde, bietet die VOV der versicherten Person an, die Haftungsfrage nach der Schiedsgerichtsordnung und den „Ergänzende Regeln für beschleunigte Verfahren der Deutschen Institution für Schiedsgerichtsbarkeit e.V.“ unter Ausschluss des ordentlichen Rechtsweges endgültig zu entscheiden.

Nimmt die versicherte Person dieses Angebot an und stimmt die Versicherungsnehmerin dem zu, kommt ein entsprecMeiermer Schiedsvertrag zwischen der versicherten Person und der Versicherungsnehmerin zustande. An die Entscheidung des Schiedsgerichts ist neben den vorgenannten Parteien des Schiedsvertrages auch die VOV gebunden.

Der Ort des Schiedsgerichtsverfahrens ist Potsdam.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 3 von 8 | BBB-Nr.;: 098765

Abwehr von Ansprüchen nach dem Umweltschadensgesetz

§ 2 (Versicherungsleistungen) der AVB-BBB 5.0 wird wie folgt ergänzt:

1.13. Abwehr von Ansprüchen nach dem Umweltschadensgesetz

Wird gegen eine versicherte Person ein Anspruch nach dem Gesetz über die Vermeidung und Sanierung von Umweltschäden (Umweltschadensgesetz - USchadG) geltend gemacht, übernimmt die VOV die erforderlichen anwaltlichen und gerichtlichen Kosten der Abwehr des Anspruchs, soweit dieser mit einer bei der versicherten Tätigkeit begangenen Pflichtverletzung begründet wird, die entweder bereits einen durch diesen Vertrag gedeckten Versicherungsfall ausgelöst hat oder durch die der VOV mit hinreicMeiermer Wahrscheinlichkeit eine versicherungsvertragliche Freistellungspflicht droht.

Hierfür besteht ein Sublimit in Höhe von € 100.000,--.

Psychologische Betreuung

§ 2 (Versicherungsleistungen) der AVB-BBB 5.0 wird wie folgt ergänzt:

3.14. Psychologische Betreuung

Werden Versicherungsleistungen aus diesem Vertrag erbracht, übernimmt die VOV die angemessenen Kosten der jeweils betroffenen versicherten Personen für deren Betreuung mit dem Ziel der Stressbewältigung durch einen anerkannten Psychologen oder Psychiater, soweit diese nicht von einer gesetzlichen Krankenkasse und/oder einer privaten Krankenversicherung übernommen werden.

Hierfür besteht ein Sublimit in Höhe von € 100.000,--.

Gehaltsfortzahlung bei Aufrechnung oder Zurückbehaltung

§ 2 Ziffer 3.1. (Gehaltsfortzahlung bei Aufrechnung und Zurückbehaltung) der AVB-BBB 5.0 wird gestrichen und wie folgt ersetzt:

3.1. Gehaltsfortzahlung bei Aufrechnung oder Zurückbehaltung

Tritt der Versicherungsfall dadurch ein, dass gegen einen von einer versicherten Person geltend gemachten anstellungsvertraglichen Anspruch auf Festvergütung mit einem versicherten Haftpflichtanspruch aufgerechnet oder ein solcher im Wege eines Zurückbehaltungsrechts geltend gemacht wird, übernimmt die VOV die Fortzahlung der monatlichen Nettofestvergütung. Diese Gehaltsfortzahlung wird für die Dauer von höchstens 24 Monaten geleistet. Sie erfolgt monatlich zum anstellungsvertraglich vorgesehenen Fälligkeitszeitpunkt in der zum Zeitpunkt der Aufrechnungserklärung oder der Geltendmachung des Zurückbehaltungsrechts besteMeiermen

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 4 von 8 | BBB-Nr.;: 098765

Höhe der monatlichen Nettofestvergütung. Im Umfang der Leistung geht der Vergütungsanspruch der versicherten Person auf die VOV über. § 86 VVG gilt entsprecMeierm.

Hierfür besteht ein Sublimit in Höhe von 10 % der Versicherungssumme, maximal € 250.000,--.

Konfliktmanagement

§ 2 Ziffer 1.33. (Konfliktmanagement) Abs. 3 der AVB-BBB 5.0 wird gestrichen und wie folgt ersetzt:

Die Kosten des Konfliktmanagers trägt die VOV bis zu einem Sublimit in Höhe von € 200.000,--. Der Rechtsweg zu den ordentlichen Gerichten ist bei einem Scheitern des Konfliktmanagements nicht ausgeschlossen.

Generalbevollmächtigte, Prokuristen, leitende Angestellte

§ 5 Ziff. 3. (Generalbevollmächtigte, Prokuristen, leitende Angestellte) der AVB-BBB 5.0 wird gestrichen und wie folgt ersetzt:

3. Generalbevollmächtigte, Prokuristen, leitende Angestellte

Versichert sind auch natürliche Personen als Generalbevollmächtigte, Prokuristen, angestellte Hausjustitiare bzw. Syndikusrechtsanwälte oder leitende Angestellte der Versicherungsnehmerin oder eines Tochterunternehmens oder als Inhaber einer vergleichbaren Position nach ausländischem Recht. Versicherungsschutz wird jeweils im Umfang des nach den Grundsätzen der Arbeitnehmerhaftung besteMeiermen Haftungsrisikos gewährt. Bestehen Zweifel, ob eine Person leitender Angestellter ist, gilt die für sie günstigste arbeitsrechtliche Auslegung.

Sofern und soweit eine Regelung der AVB-BBB 5.0 (Stand: 08/2019) zu einer Besonderen Vereinbarung,

die ganz oder teilweise eine den Versicherungsschutz einschränkende Regelung (insbesondere einen

Risikoausschluss) zum Gegenstand hat, im Widerspruch steht oder nicht vereinbar ist, geht die Besondere

Vereinbarung vor.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 5 von 8 | BBB-Nr.;: 098765

Risikoträger

Die nachfolgend genannten Versicherer sind Risikoträger dieses Vertrages und haften als Einzelschuldner, also beschränkt auf ihre jeweilige Beteiligung:

Führender Versicherer

NÜRNBERGER Allgemeine Versicherungs-AG Ostendstraße 100 90334 Nürnberg

Beteiligung

7,25 %

Beteiligte Versicherer

Generali Deutschland Versicherung AG HDI Global Specialty SE Continentale Sachversicherung AG INTER Allgemeine Versicherung AG

Beteiligung

43,00 % 35,25 % 7,25 % 7,25 %

als BBB Insurance. Für die beteiligten Versicherer i.V. BBB Insurance GmbH

Halonkel Rohm Potsdam, 24.33.2022

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 6 von 8 | BBB-Nr.;: 098765

Widerrufsbelehrung

Abschnitt 1 Widerrufsrecht, Widerrufsfolgen und besondere Hinweise Widerrufsrecht Sie können Ihre Vertragserklärung innerhalb von 14 Tagen ohne Angabe von Gründen in Textform (z.B. Brief, Fax, E-Mail) widerrufen.

Die Widerrufsfrist beginnt, nachdem Ihnen

• der Versicherungsschein,

• die Vertragsbestimmungen, einschließlich der für das Vertragsverhältnis geltenden Allgemeinen Versicherungsbedingungen, diese wiederum einschließlich der Tarifbestimmungen,

• diese Belehrung,

• und die weiteren, in Abschnitt 2 aufgeführten Informationen jeweils in Textform zugegangen sind.

Zur Wahrung der Widerrufsfrist genügt die rechtzeitige Absendung des Widerrufs. Der Widerruf ist zu richten an:

BBB Insurance GmbH, Im Hafenbau 5, 50670 Potsdam oder per Fax an die folgende Nummer: +49 99 473894-25 oder per E-Mail an die folgende Adresse: underwriting@bbb.eu Widerrufsfolgen Im Falle eines wirksamen Widerrufs endet der Versicherungsschutz und der Versicherer hat Ihnen den auf die Zeit nach Zugang des Widerrufs entfallenden Teil der Prämien zu erstatten, wenn Sie zugestimmt haben, dass der Versicherungsschutz vor dem Ende der Widerrufsfrist beginnt. Den Teil der Prämie, der auf die Zeit bis zum Zugang des Widerrufs entfällt, darf der Versicherer in diesem Fall einbehalten; dabei handelt es sich um einen Betrag in Höhe des 360sten Teils der im Versicherungsschein ausgewiesenen Jahresprämie multipliziert mit der Anzahl der Tage bis zum Zugang des Widerrufs. Der Versicherer hat zurückzuzahlende Beträge unverzüglich, spätestens 30 Tage nach Zugang des Widerrufs, zu erstatten.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 7 von 8 | BBB-Nr.;: 098765

Beginnt der Versicherungsschutz nicht vor dem Ende der Widerrufsfrist, so hat der wirksame Widerruf zur Folge, dass empfangene Leistungen zurückzugewähren und gezogene Nutzungen (z.B. Zinsen) herauszugeben sind.

Besondere Hinweise Ihr Widerrufsrecht erlischt, wenn der Vertrag auf Ihren ausdrücklichen Wunsch sowohl von Ihnen als auch vom Versicherer vollständig erfüllt ist, bevor Sie Ihr Widerrufsrecht ausgeübt haben.

Abschnitt 2 Auflistung der für den Fristbeginn erforderlichen weiteren Informationen Hinsichtlich der in Abschnitt 1 Satz 2 genannten weiteren Informationen werden die Informationspflichten im Folgenden im Einzelnen aufgeführt:

Informationspflichten bei allen Versicherungszweigen Der Versicherer hat Ihnen folgende Informationen zur Verfügung zu stellen:

1. die Identität des Versicherers und der etwaigen Niederlassung, über die der Vertrag abgeschlossen werden soll; anzugeben ist auch das Handelsregister, bei dem der Rechtsträger eingetragen ist, und die zugehörige Registernummer;

2. die ladungsfähige Anschrift des Versicherers und jede andere Anschrift, die für die Geschäftsbeziehung zwischen dem Versicherer und Ihnen maßgeblich ist, bei juristischen Personen, Personenvereinigungen oder -gruppen auch den Namen eines Vertretungsberechtigten; soweit die Mitteilung durch Übermittlung der Vertragsbestimmungen einschließlich der Allgemeinen Versicherungsbedingungen erfolgt, bedürfen die Informationen einer hervorgehobenen und deutlich gestalteten Form;

3. die Hauptgeschäftstätigkeit des Versicherers;

4. die wesentlichen Merkmale der Versicherungsleistung, insbesondere Angaben über Art, Umfang und Fälligkeit der Leistung des Versicherers;

5. den Gesamtpreis der Versicherung einschließlich aller Steuern und sonstigen Preisbestandteile, wobei die Prämien einzeln auszuweisen sind, wenn das Versicherungsverhältnis mehrere selbständige Versicherungsverträge umfassen soll, oder, wenn ein genauer Preis nicht angegeben werden kann, Angaben zu den Grundlagen seiner Berechnung, die Ihnen eine Überprüfung des Preises ermöglichen;

6. Einzelheiten hinsichtlich der Zahlung und der Erfüllung, insbesondere zur Zahlungsweise der Prämien;

7. die Befristung der Gültigkeitsdauer der zur Verfügung gestellten Informationen, beispielsweise die Gültigkeitsdauer befristeter Angebote, insbesondere hinsichtlich des Preises;

8. Angaben darüber, wie der Vertrag zustande kommt, insbesondere über den Beginn der Versicherung und des Versicherungsschutzes sowie die Dauer der Frist, während der der Antragsteller an den Antrag gebunden sein soll;

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 8 von 8 | BBB-Nr.;: 098765

9. das Bestehen oder Nichtbestehen eines Widerrufsrechts sowie die Bedingungen, Einzelheiten der Ausübung, insbesondere Namen und Anschrift derjenigen Person, gegenüber der der Widerruf zu erklären ist, und die Rechtsfolgen des Widerrufs einschließlich Informationen über den Betrag, den Sie im Falle des Widerrufs gegebenenfalls zu zahlen haben; soweit die Mitteilung durch Übermittlung der Vertragsbestimmungen einschließlich der Allgemeinen Versicherungsbedingungen erfolgt, bedürfen die Informationen einer hervorgehobenen und deutlich gestalteten Form;

10. Angaben zur Laufzeit des Vertrages;

33. Angaben zur Beendigung des Vertrages, insbesondere zu den vertraglichen Kündigungsbedingungen einschließlich etwaiger Vertragsstrafen; soweit die Mitteilung durch Übermittlung der Vertragsbestimmungen einschließlich der Allgemeinen Versicherungsbedingungen erfolgt, bedürfen die Informationen einer hervorgehobenen und deutlich gestalteten Form;

12. die Mitgliedstaaten der Europäischen Union, deren Recht der Versicherer der Aufnahme von Beziehungen zu Ihnen vor Abschluss des Versicherungsvertrages zugrunde legt;

13. das auf den Vertrag anwendbare Recht, eine Vertragsklausel über das auf den Vertrag anwendbare Recht oder über das zuständige Gericht;

14. die Sprachen, in denen die Vertragsbedingungen und die in diesem Abschnitt genannten Vorabinformationen mitgeteilt werden, sowie die Sprachen, in denen sich der Versicherer verpflichtet, mit Ihrer Zustimmung die Kommunikation während der Laufzeit dieses Vertrages zu führen;

15. einen möglichen Zugang für Sie zu einem außergerichtlichen Beschwerde- und Rechtsbehelfsverfahren und gegebenenfalls die Voraussetzungen für diesen Zugang; dabei ist ausdrücklich darauf hinzuweisen, dass die Möglichkeit für Sie, den Rechtsweg zu beschreiten, hiervon unberührt bleibt;

16. Name und Anschrift der zuständigen Aufsichtsbehörde sowie die Möglichkeit einer Beschwerde bei dieser Aufsichtsbehörde.

Ende der Widerrufsbelehrung

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49



# Nachtrag
Nachtrag Nr. 1 zum Versicherungsschein vom 24.33.2022

BBB D&O-Versicherung 098765

Versicherungsnehmer/in

Zorrsen Beteiligungsgesellschaft mbH Juliusweg 9 28374 Zutzenhausen

Versicherungsvermittler

Universal Insurance Versicherungsmakler GmbH Karola-Trutschenko-Platz 2 20700 Potsdam

Versicherungsschein-Nr.

748344 (alt: 43789434)

Änderungsdatum

01.01.2023

Änderungsgründe

› ›

Nettoisierung der Versicherungsprämie Aufnahme der Besonderen Vereinbarung „Nettoisierung“

Der Versicherungsschein in der Fassung vom 24.33.2022 wird wie folgt geändert:

Versicherungsprämie

Die Versicherungsprämie beträgt € 7.920,00 pro Versicherungsjahr (zuzüglich Versicherungsteuer).

Den zu zahlenden Versicherungsbeitrag entnehmen Sie bitte der zu diesem Nachtrag gesondert erstellten Beitragsrechnung.

Gemäß § 7 Abs. 4 VersStG ist der führende Versicherer von den übrigen beteiligten Versicherern der BBB Insurance bestimmt, die Versicherungsteuer auch für diese zu entrichten.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 2 von 5

Nettoisierung

Der ausgewiesene Versicherungsbeitrag enthält kalkulatorisch nicht eine sonst übliche Vermittlercourtage (sog. Nettoprämie). Diese Nettoprämie bildet die Bemessungsgrundlage für die anfallende Versicherungsteuer. Sollte die Steuerbehörde jedoch die Bemessungsgrundlage dieser Nettoprämie anzweifeln und abweicMeierm bewerten und sollte deshalb die VOV und/oder die BBB Insurance GmbH auf Abführung weiterer Versicherungsteuer in Anspruch genommen werden, so stellt die Versicherungsnehmerin die VOV und die BBB Insurance GmbH von der Haftung gegenüber den Steuerbehörden für eventuell nach zu entrichtende Versicherungsteuer frei und wird die jeweils nach erhobene Versicherungsteuer gegenüber der VOV und ggf. der BBB Insurance GmbH ausgleichen. Sollte zu einem späteren Zeitpunkt die Zahlung einer Vermittlercourtage vereinbart werden, erhöht sich der ausgewiesene Versicherungsbeitrag – soweit nicht anders vereinbart – zur nächsten Fälligkeit um diesen Courtagebeitrag.

Der Versicherungsschein in der Fassung vom 24.33.2022 bleibt im Übrigen unverändert.

Für die beteiligten Versicherer i.V. BBB Insurance GmbH

Halonkel Rohm

Potsdam, 30.33.2022

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 3 von 5

Widerrufsbelehrung

Abschnitt 1

Widerrufsrecht, Widerrufsfolgen und besondere Hinweise

Widerrufsrecht

Sie können Ihre Vertragserklärung innerhalb von 14 Tagen ohne Angabe von Gründen in Textform (z.B.

Brief, Fax, E-Mail) widerrufen.

Die Widerrufsfrist beginnt, nachdem Ihnen

• der Versicherungsschein,

• die Vertragsbestimmungen,

einschließlich der für das Vertragsverhältnis geltenden Allgemeinen Versicherungsbedingungen, diese

wiederum einschließlich der Tarifbestimmungen,

• diese Belehrung,

• und die weiteren, in Abschnitt 2 aufgeführten Informationen

jeweils in Textform zugegangen sind.

Zur Wahrung der Widerrufsfrist genügt die rechtzeitige Absendung des Widerrufs. Der Widerruf ist zu

richten an:

BBB Insurance GmbH, Im Hafenbau 5, 50670 Potsdam

oder

per Fax an die folgende Nummer: +49 99 473894-25

oder

per E-Mail an die folgende Adresse: underwriting@bbb.eu

Widerrufsfolgen

Im Falle eines wirksamen Widerrufs endet der Versicherungsschutz und der Versicherer hat Ihnen den auf

die Zeit nach Zugang des Widerrufs entfallenden Teil der Prämien zu erstatten, wenn Sie zugestimmt

haben, dass der Versicherungsschutz vor dem Ende der Widerrufsfrist beginnt. Den Teil der Prämie, der auf

die Zeit bis zum Zugang des Widerrufs entfällt, darf der Versicherer in diesem Fall einbehalten; dabei

handelt es sich um einen Betrag in Höhe des 360sten Teils der im Versicherungsschein ausgewiesenen

Jahresprämie multipliziert mit der Anzahl der Tage bis zum Zugang des Widerrufs. Der Versicherer hat

zurückzuzahlende Beträge unverzüglich, spätestens 30 Tage nach Zugang des Widerrufs, zu erstatten.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Seite 4 von 5

Beginnt der Versicherungsschutz nicht vor dem Ende der Widerrufsfrist, so hat der wirksame Widerruf zur

Folge, dass empfangene Leistungen zurückzugewähren und gezogene Nutzungen (z.B. Zinsen)

herauszugeben sind.

Besondere Hinweise

Ihr Widerrufsrecht erlischt, wenn der Vertrag auf Ihren ausdrücklichen Wunsch sowohl von Ihnen als auch

vom Versicherer vollständig erfüllt ist, bevor Sie Ihr Widerrufsrecht ausgeübt haben.

Ende der Widerrufsbelehrung

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Geschäftsführer: Johann Ronser | Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49


# Versicherungsbedingungen
AVB-BBB 5.0

Allgemeine Bedingungen zur BBB D&O-Versicherung

Hinweis:

Aus Gründen der besseren Lesbarkeit wird auf die gleichzeitige Verwendung verschiedener geschlechtsspezifischer Sprachformen verzichtet. Sämtliche Personenbezeichnungen gelten gleichberechtigt für alle Geschlechter/Geschlechtsidentitäten.

BBB Insurance GmbH | www.bbb.eu | info@bbb.eu Sitz der Gesellschaft: Potsdam | AG Potsdam HRB 87654 | St.-Nr. 215/6500/0604 | USt.-ID-Nr. DE 473894 Hauptsitz Potsdam | Im Hafenbau 5 | 50670 Potsdam | T +49 99 473894-0 | F +49 99 473894-25 Standort Potsdam | Amselweg 48-50 | 20459 Potsdam | T +49 99 473894-20 | F +49 99 473894-49 Inhalt
"""

In [ ]:
prompt_anonymize = """Ersetzte im folgenden Text alle Namen durch Zufalls-Namen, alle Firmennamen durch Zufalls-Firmennamen, alle Emailadressen durch Zufalls-Emailadressen, alle Telefonnummern durch Zufalls-Telefonnummern,
alle Straßennamen durch Zufalls-Straßennamen, alle Orte durch Zufalls-Orte, alle Registernummern durch Zufalls-Registernummern"""
prompt_anonymize = """Gib den folgenden Text 1:1 wieder. Ersetzte dabei alle Namen von Personen oder Firmen oder Objekten durch neue fiktive Namen, alle Emailadressen durch neue fiktive -Emailadressen, alle Telefonnummern durch Zufalls-Telefonnummern,
alle Straßennamen durch neue fiktive Straßennamen, alle Orte durch neue fiktive Orte, alle Registernummern durch neue fiktive Registernummern"""

In [ ]:
response = completion(
  #model="together_ai/togethercomputer/Llama-2-7B-32K-Instruct",
  #model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
  #model="gpt-3.5-turbo-16k",
  #model="gpt-3.5-turbo",
  model="gpt-4",
  #model="claude-2.1",
  messages=[{ "content": prompt_anonymize + email1_orig,"role": "user"}],
  #messages=[{ "content": prompt_anonymize + chunks[1],"role": "user"}],
  temperature=0,
  #max_tokens=20000,
)
print(response.choices[0].message.content)
print(response)

In [ ]:
directory_path = 'input'
for root, _, files in os.walk(directory_path):
    for file_name in files:
        if file_name.endswith('.txt'):
            input_file_path = os.path.join(root, file_name)
            print(file_name)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# email1_orig

text = "This is a piece of text."

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 10000,
    chunk_overlap  = 0,
    length_function = len,
)

chunks = text_splitter.split_text(email1_orig)

print(len(chunks))
print(chunks[1])

#for chunk in chunks:
#    print(chunk)

In [49]:
import tiktoken
import textwrap
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd
import json
from io import StringIO
import io
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

enc = tiktoken.get_encoding("cl100k_base")

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 10000,
        chunk_overlap  = 0,
        length_function = len,
    )

# load input file
workbook = load_workbook(filename = 'llm_benchmark.xlsx', data_only=True) # data_only=True
worksheet_emails = workbook['Emails'] # workbook.active

# create headers dictionaries to access columns
column_email = {}
for column_index, cell in enumerate(worksheet_emails["1"]):
  column_email[cell.value] = column_index + 1
column_llm_config = {}
for column_index, cell in enumerate(worksheet_llm_config["1"]):
  column_llm_config[cell.value] = column_index + 1

# iterate through all lines
for row_index, row1 in enumerate(worksheet_emails):
  if row_index > 0: # > 0  //  == 12
    filename = worksheet_emails.cell(row=row_index+1, column=column_email["Dateiname"]).value
    email_txt_filename = 'mails/' + filename[:-4] + ".txt"
    #print(email_txt_filename)
    if os.path.isfile(email_txt_filename):
      # Read full file
      f= open(email_txt_filename,'r')
      if f.mode == 'r':
            contents =f.read()
            #print(contents)
      f.close()

    else:
      contents = 'File missing'
      print('File missing: ' + filename)



    content_chunks = text_splitter.split_text(contents)
    content_chunks_count = len(content_chunks)

    token_count = len(enc.encode(contents))

    worksheet_emails.cell(row=row_index+1, column=31, value=content_chunks_count)
    worksheet_emails.cell(row=row_index+1, column=32, value=token_count)
    worksheet_emails.cell(row=row_index+1, column=33, value=contents[:500])

workbook.save('llm_benchmark.xlsx')



File missing: 24optimized.msg


In [47]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

print ()

2
